# Kurs 7

## Metaprogramming

Der Begriff Metaprogramming heißt soviel wie: Code, der Code generiert. Eine der schönen Eigenschaften von Julia ist, dass Metaprogramming hier sehr elegant funktioniert. In Sprachen wie C++ ist das immer absolut nervig, weil man sozusagen einen extra Überbau über die Sprache benötigt, der den Code manipuliert, *bevor* er kompiliert wird. Insbesondere also *bevor* der kompilierte Code ausgeführt werden kann. Das sind dann sogenannte preprocessor directives.

In Julia verwendet man stattdessen sogenannte Makros. Der große Vorteil gegenüber zu etwa C++ ist, dass man hier durch den JIT beispielsweise einen Variablennahmen mit regex *zur Laufzeit* erzeugen und dann im nächsten Schritt ein Makro zur Generierung von neuem Code mit dem Variablennahmen verwenden kann. Das heißt im Klartext: Diese strikte Trennung von Codeerzeugung und Codeausführung gibt es nicht mehr.

Offtopic erklärt das ganz gut den (mehr oder weniger lustigen) Witz vom Julia-Mitentwickler Jeff Bezanson, der Name Julia könne auch für "Jeff's uncommon lisp is automated" stehen. Lisp ist nämlich eine Programmiersprache deren Stärke insbesondere das Metaprogramming ist. Julias Funktionalität in diesem Bereich kommt also nicht von ungefähr.

### Expressions

Um diese kleine Vorbemerkung ein bisschen plastischer zu machen, begeben wir uns nun auf eine Mini-Reise durch die Ausführungschritte von Julia-Code (ausführlichere Infos [hier](https://docs.julialang.org/en/v1/devdocs/eval/)). Wenn wir in Julia eine Zeile Code eingeben, dann haben wir zunächst ja einfach nur Text, also einen String. Dieser wird dann in eine `Expr` (expression) umgewandelt. Das nennt man `parsen`:

In [ ]:
input_expr = Meta.parse("1 + 2")

In [ ]:
input_expr |> typeof

Unsere expression wird schlussendlich in ausführbaren Maschinencode übersetzt und ausgeführt:

In [ ]:
eval(input_expr) # Nebenbemerkung: eval läuft IMMER in der global scope!

Wenn wir jetzt aber nochmal einen Schritt zurück machen, dann sehen wir, dass in der `Expr` unser Eingabetext in eine hierarchische Baumstruktur gebracht wurde:

In [ ]:
input_expr |> dump

In [ ]:
input_expr.head

In [ ]:
input_expr.args

Dabei gibt es immer einen *head* und einen *body*, also sozusagen Stamm und Äste. In unserem Fall ist der head `:call`, weil wir den Operator `+` auf die Inputs `1` und `2` anwenden. Betrachten wir ein etwas schwierigeres Beispiel:

In [ ]:
advanced_expr = :(f(1) + 2) # wir erstellen uns direkt eine Expr

Hier stellen wir nun fest, dass anstelle der `1` eine weitere (geschachtelte) Expression getreten ist:

In [ ]:
advanced_expr |> dump

Ganz analog ist in dieser der head `:call`, weil wir wir einen Funktionsaufruf (call) `f(x)` haben.

In [ ]:
advanced_expr.args[2].head

Außerdem können wir (ähnlich wie bei Strings) *interpolation* nutzen:

In [ ]:
x = 2
:(sqrt($x))

### Mickey Mouse Example

Wofür brauchen wir nun Makros? Nehmen wir einmal an, wir würden gerne die Variablen ```a``` bis ```z``` so belegen, dass ```a = 1, ..., z = 26```. Dann wollen wir das natürlich nicht händisch ausschreiben, sondern möglichst arbeitssparend tun. Dafür können wir das Makro `@eval` nutzen:

In [ ]:
# Anfangsbuchstabe (character) ist a
ch = 'a'

for i in 1:26
    # wir nutzen nun das Makro, weil wir so nicht extra eine Expr erzeugen müssen
    @eval $(Symbol(ch)) = $i # Interpolation mit $
    ch += 1
end

y

Jetzt kann man sich fragen: Warum genau sollte man das tun wollen? Variablen derart zu benennen ist wahrscheinlich Quatsch, stattdessen würde man wohl eher mit dem Array `collect(1:26)` arbeiten. Eine sinnvolle Anwendung findet man aber zum Beispiel beim Makro `@assert`:

In [ ]:
@macroexpand @assert (a == 1) "a ist nicht 1!!"

### Eigene Makros

Grundsätzlich gibt es viele Zwecke für Metaprogramming, aber in der Regel fällt die Nutzung von Makros in eine von zwei Kategorien:

- Wir wollen neue "language features" implementieren
- Wir wollen syntactic sugar (hübschere Schreibweisen)

Betrachten wir ein einfaches Makro, das quadriert:

In [ ]:
macro squared(ex)
    return :($(ex) * $(ex))
end

@squared(2)

In [ ]:
@macroexpand @squared(2)

Auf den ersten Blick sieht alles gut aus, allerdings gibt es ein Problem:

In [ ]:
function foo()
    x = 2
    return @squared x
end

In [ ]:
foo()

Unser Makro interpretiert `x` als eine globale Variable und nicht als lokale Variable in der Funktion! Der Versuch, dort den entsprechenden Wert reinzukopieren, schlägt also fehl.

In [ ]:
@code_lowered foo()

Dieses Problem lässt sich durch *escaping* beheben. Wir teilen unserem Makro mit, dass `ex` vom Compiler in Ruhe gelassen werden sollte.

In [ ]:
macro squared(ex)
    return :($(esc(ex)) * $(esc(ex)))
end

function foo()
    x = 2
    return @squared x
end

In [ ]:
foo()

Hier ist ein weiteres Beispiel, wo escaping notwendig ist; der Compiler ersetzt Variablen- und Funktionsnamen stets durch eigene. Das wollen wir hier nicht.

In [ ]:
macro trick(expr)
    trick_msg = :(println("Wir benutzen eine trickreiche Funktion!"))
    return Expr(:block, trick_msg, expr)
end

In [ ]:
@trick bar(x) = 1

In [ ]:
macro trick(expr)
    trick_msg = :(println("Wir benutzen eine trickreiche Funktion!"))
    return esc(Expr(:block, trick_msg, expr))
end

In [ ]:
@trick foobar(x) = 1

Wichtige Makros sind außerdem

- `@time`
- `@macroexpand`
- `@which`
- `@edit`


### Ausblick: Loop unrolling (SIMD)

Ein anwendungsnäheres Beispiel als oben wäre etwa das sogenannte loop unrolling. Sagen wir, wir haben einen Vektor, der als Länge ein Vielfaches von 4 hat. Nun muss man wissen: Viele Hardwarearchitekturen haben beschleunigte Operationen implementiert, wenn man viele punktweise (gleiche) Operationen durchführt. Das nennt man auch SIMD (single instruction, multiple data).
Weil Loops erst zur Laufzeit ausgeführt werden, weiß der Kompiler aber nicht immer, dass wir hier so etwas vorliegen haben. Wir können uns also ein Makro schreiben, das aus
```
for i=1:4n
    c[i] = a[i]*b[i]
end
```
den Code
```
for i=1:n
    c[i*4]     = a[i*4]    *b[i*4]
    c[i*4 + 1] = a[i*4 + 1]*b[i*4 + 1]
    c[i*4 + 2] = a[i*4 + 2]*b[i*4 + 2]
    c[i*4 + 3] = a[i*4 + 3]*b[i*4 + 3]
end
```
generiert.

Auf diese Weise weiß der Compiler, dass hier sozusagen 4 mal die gleiche Operation durchgeführt wird und kann diesen Prozess optimieren. Am einfachsten geht dies mit dem Makro `Base.Cartesian.@nexprs`.

In [ ]:
@macroexpand Base.Cartesian.@nexprs 4 j -> c[i * 4 + (j - 1)] = a[i * 4 + (j - 1)] * b[i * 4 + (j - 1)]

### Ausblick: Eigene Syntax bzw. Modelldefinitionen

Ebenfalls praktisch sind Makros beim Erstellen von eigener (Modell-)Syntax. Beispielweise möchte man oft Parameterwerte in einer `Dict`-ähnlichen Struktur halten:

In [ ]:
my_parameters = Dict(
    "ε" => 0.9,
    "δ" => 0.5,
    "α₁" => 0.2,
    "α₂" => 0.3
)

Die Anführungszeichen, der Pfeil `=>` und das Komma am Ende sind aber nervig zu tippen und ein `=` wäre auch aus mathematische Sicht schöner. Außerdem hätte man vielleicht gerne einen eigenen Datentyp, damit der User nicht ein fremdes, potenziell sinnfreies `Dict` an Funktionen übergeben kann. Dafür definieren wir uns (welch Überraschung) ein Makro:

In [ ]:
macro parameters(block)
    exprs = block.args

    # Filtere auf Zuweisungen
    assignments = filter(e -> isa(e, Expr) && e.head == :(=), exprs)

    # Extrahiere Variablennamen und deren Werte als Symbole
    pairs = [Expr(:call, :(=>), :(Symbol($("$(a.args[1])"))), a.args[2]) for a in assignments]

    return esc(Expr(:call, :(Dict), pairs...)) # hier möglicherweise ein anderer Datentyp als Dict
end

my_parameters = @parameters begin
    ε = 0.9
    δ = 0.5
    α₁ = 0.2
    α₂ = 0.3
end